In [12]:
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchgeo.datasets import unbind_samples
from torchgeo.datasets import RasterDataset
from torchgeo.samplers import RandomGeoSampler
from torchgeo.datasets import RasterDataset
import torch
from torchvision.transforms import Compose
from torchgeo.samplers import RandomGeoSampler
import random
import torchvision.transforms.functional as F
from torchvision.transforms import v2
import torch.nn as nn

In [13]:
class RandomHorizontalFlip(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def forward(self, sample):
        if random.random() < self.p:
            if 'image' in sample:
                sample['image'] = F.hflip(sample['image'])
            if 'mask' in sample:
                sample['mask'] = F.hflip(sample['mask'])
        return sample

class RandomVerticalFlip(nn.Module):
    """Randomly flips the image and mask vertically with a given probability.
    Args:
        p (float): Probability of flipping the image and mask. Default is 0.5
    """
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def forward(self, sample):
        if random.random() < self.p:
            if 'image' in sample:
                sample['image'] = F.vflip(sample['image'])
            if 'mask' in sample:
                sample['mask'] = F.vflip(sample['mask'])
        return sample

class RandomRotation(nn.Module):
    """Randomly rotates the image and mask by a random angle within a specified range.
    Args:
        degrees (float or tuple): Range of degrees to select from. If a single float is provided, the range will be (-degrees, +degrees).
    """
    def __init__(self, degrees):
        super().__init__()
        self.degrees = degrees

    def forward(self, sample):
        angle = random.uniform(-self.degrees, self.degrees)
        if 'image' in sample:
            sample['image'] = F.rotate(sample['image'], angle)
        if 'mask' in sample:
            sample['mask'] = F.rotate(sample['mask'], angle, interpolation=F.InterpolationMode.NEAREST)
        return sample

transforms = Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),

    # RandomHorizontalFlip(p=0.5),
    # RandomVerticalFlip(p=0.5),
    # RandomRotation(degrees=15),
    # v2.ToTensor(),
    # Lambda(lambda x: torch.nan_to_num(x, nan=0.0)),

    v2.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
    v2.RandomAutocontrast(p=0.3),
    v2.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    v2.RandomApply([v2.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0))], p=0.3),
    v2.Normalize(mean=[0.36576813, 0.3658635, 0.3988132],
              std=[0.16295877, 0.17293826, 0.15380774]),
])

class NoDataAware_RandomSampler(RandomGeoSampler):
    def __init__(self, dataset, size, length, nodata_value=0, max_nodata_ratio=0.4, **kwargs):
        super().__init__(dataset, size, length, **kwargs)
        self.nodata_value = nodata_value
        self.max_nodata_ratio = max_nodata_ratio
        self.dataset = dataset
    
    def __iter__(self):
        generated = 0
        while generated < self.length:
            # Get a random query from parent sampler
            query = next(super().__iter__())
            
            try:
                # Check the sample for nodata
                sample = self.dataset[query]
                if 'mask' in sample:
                    mask = sample['mask']
                    nodata_ratio = (mask == self.nodata_value).float().mean().item()
                    # Skip if too much nodata
                    if nodata_ratio >= self.max_nodata_ratio:
                        continue
                
                yield query
                generated += 1
            except Exception as e:
                # Skip problematic samples
                print(f"Skipping sample {query} due to error: {e}")
                continue

class Liss4(RasterDataset):
    filename_glob = '*.tif'
    filename_regex = r'^.{3}(?P<date>\d{2}[A-Z]{3}\d{4})'
    date_format = '%d%b%Y'
    single_band = False
    is_image = True
    separate_files = False
    # transforms = transforms

    def __getitem__(self, query):
        sample = super().__getitem__(query)

        # if "imgs" in sample:
        image = sample["image"]
        image = torch.nan_to_num(image, nan=0.0)
        sample['chn_ids'] = torch.tensor([842, 665, 560])
        sample["imgs"] = image
        return sample

class Liss4_GT(RasterDataset):
    filename_glob = '*.tif'
    filename_regex = r'^.{3}(?P<date>\d{2}[A-Z]{3}\d{4})'
    date_format = '%d%b%Y'
    single_band = False
    is_image = False
    separate_files = False

    def __getitem__(self, query):
        sample = super().__getitem__(query)
        mask = sample["mask"]
        mask = torch.nan_to_num(mask, nan=0.0)
        mask = torch.where(mask < 0, torch.tensor(0, dtype=torch.int64), mask)
        sample["mask"] = mask
        return sample


train_img = Liss4("../data/unprocessed_data/train/img")
train_label = Liss4_GT("../data/unprocessed_data/train/label")

val_img = Liss4("../data/unprocessed_data/test/img")
val_label = Liss4_GT("../data/unprocessed_data/test/label")


train_dataset = train_img & train_label
train_dataset.transforms = transforms

val_dataset = val_img & val_label

In [14]:
from torchgeo.samplers import RandomGeoSampler, GridGeoSampler
from torch.utils.data import DataLoader
from torchgeo.datasets import stack_samples, unbind_samples

# train_sampler = NoDataAware_RandomSampler(train_dataset, size=224, length=, nodata_value=0, max_nodata_ratio=0.4)
train_sampler = RandomGeoSampler(train_dataset, size=224, length=20)
train_dataloader = DataLoader(train_dataset, batch_size=1, sampler=train_sampler, collate_fn=stack_samples)


val_sampler = GridGeoSampler(val_dataset, size=224, stride=210)
val_dataloader = DataLoader(val_dataset, batch_size=1, sampler=val_sampler, collate_fn=stack_samples)

In [15]:
def visualize(dataloader):
    for batch in dataloader:
        sample = unbind_samples(batch)[0]
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        ax[0].imshow(sample['image'].permute(1, 2, 0).numpy())
        ax[0].set_title('Image')
        print(np.unique(sample['mask']))
        ax[1].imshow(sample['mask'].squeeze().numpy(), cmap='gray')
        ax[1].set_title('Mask')
        
        plt.axis('off')
        plt.tight_layout()
        plt.show()

In [ ]:
visualize(val_dataloader)